In [1]:
import requests
from pyquery import PyQuery as pq
import pandas as pd

In [2]:
player_df = pd.read_csv("processed_data/bref_player_info_and_id_06_06_17.csv")

In [3]:
player_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4460 entries, 0 to 4459
Data columns (total 12 columns):
Player        4460 non-null object
From          4460 non-null int64
To            4460 non-null int64
Pos           4459 non-null object
Ht            4459 non-null object
Wt            4454 non-null float64
Birth_Date    4429 non-null object
College       4169 non-null object
Link          4460 non-null object
Bref_ID       4460 non-null object
HOF           4460 non-null int64
Ht_Inches     4459 non-null float64
dtypes: float64(2), int64(3), object(7)
memory usage: 418.2+ KB


In [4]:
player_df.head()

,Player,From,To,Pos,Ht,Wt,Birth_Date,College,Link,Bref_ID,HOF,Ht_Inches
0,Alaa Abdelnaby,1991,1995,F-C,6-10,240.0,1968-06-24,Duke University,/players/a/abdelal01.html,abdelal01,0,82.0
1,Zaid Abdul-Aziz,1969,1978,C-F,6-9,235.0,1946-04-07,Iowa State University,/players/a/abdulza01.html,abdulza01,0,81.0
2,Kareem Abdul-Jabbar,1970,1989,C,7-2,225.0,1947-04-16,"University of California, Los Angeles",/players/a/abdulka01.html,abdulka01,1,86.0
3,Mahmoud Abdul-Rauf,1991,2001,G,6-1,162.0,1969-03-09,Louisiana State University,/players/a/abdulma02.html,abdulma02,0,73.0
4,Tariq Abdul-Wahad,1998,2003,F,6-6,223.0,1974-11-03,San Jose State University,/players/a/abdulta01.html,abdulta01,0,78.0


In [5]:
def scrape_cbb_id(link):
    try:
        url = "http://www.basketball-reference.com" + link
        response = requests.get(url)
        html = response.text.replace('<!--', '').replace('-->', '')
        d = pq(html)
        inner_nav_links = d("#inner_nav > ul > li.condensed.hasmore > div > ul > li > a")
        list_text = [i.text_content() for i in inner_nav_links]
        # get the index of the sublist that contains the substring about 
        # college stats
        college_text = "College Basketball at Sports-Reference.com"
        idx = [i for i, s in enumerate(list_text) if college_text in s][0]
        college_li = inner_nav_links[idx]
        link = college_li.attrib["href"]
        return link
    except Exception as e:
        return [url, e]

In [6]:
test_run = player_df.loc[:5, "Link"].apply(scrape_cbb_id)

In [7]:
test_run

0    http://www.sports-reference.com/cbb/players/al...
1    [http://www.basketball-reference.com/players/a...
2    http://www.sports-reference.com/cbb/players/ka...
3    http://www.sports-reference.com/cbb/players/ma...
4    [http://www.basketball-reference.com/players/a...
5    http://www.sports-reference.com/cbb/players/sh...
Name: Link, dtype: object

In [8]:
test_run[0]

'http://www.sports-reference.com/cbb/players/alaa-abdelnaby-1.html'

In [9]:
test_run[1]

['http://www.basketball-reference.com/players/a/abdulza01.html',
 IndexError('list index out of range')]

In [12]:
cbb_ids = player_df.Link.apply(scrape_cbb_id)

In [13]:
player_df["cbb_info"] = cbb_ids
player_df.to_csv("raw_data/cbb_id_info.csv", index=False)

In [15]:
cbb_ids.apply(type)

0        <class 'str'>
1       <class 'list'>
2        <class 'str'>
3        <class 'str'>
4       <class 'list'>
5        <class 'str'>
6        <class 'str'>
7       <class 'list'>
8       <class 'list'>
9       <class 'list'>
10       <class 'str'>
11      <class 'list'>
12       <class 'str'>
13      <class 'list'>
14       <class 'str'>
15       <class 'str'>
16      <class 'list'>
17      <class 'list'>
18       <class 'str'>
19       <class 'str'>
20       <class 'str'>
21       <class 'str'>
22       <class 'str'>
23      <class 'list'>
24       <class 'str'>
25       <class 'str'>
26       <class 'str'>
27       <class 'str'>
28       <class 'str'>
29       <class 'str'>
             ...      
4430     <class 'str'>
4431     <class 'str'>
4432     <class 'str'>
4433    <class 'list'>
4434     <class 'str'>
4435     <class 'str'>
4436     <class 'str'>
4437     <class 'str'>
4438     <class 'str'>
4439    <class 'list'>
4440    <class 'list'>
4441     <class 'str'>
4442    <cl

In [16]:
player_df.head()

,Player,From,To,Pos,Ht,Wt,Birth_Date,College,Link,Bref_ID,HOF,Ht_Inches,cbb_info
0,Alaa Abdelnaby,1991,1995,F-C,6-10,240.0,1968-06-24,Duke University,/players/a/abdelal01.html,abdelal01,0,82.0,http://www.sports-reference.com/cbb/players/al...
1,Zaid Abdul-Aziz,1969,1978,C-F,6-9,235.0,1946-04-07,Iowa State University,/players/a/abdulza01.html,abdulza01,0,81.0,[http://www.basketball-reference.com/players/a...
2,Kareem Abdul-Jabbar,1970,1989,C,7-2,225.0,1947-04-16,"University of California, Los Angeles",/players/a/abdulka01.html,abdulka01,1,86.0,http://www.sports-reference.com/cbb/players/ka...
3,Mahmoud Abdul-Rauf,1991,2001,G,6-1,162.0,1969-03-09,Louisiana State University,/players/a/abdulma02.html,abdulma02,0,73.0,http://www.sports-reference.com/cbb/players/ma...
4,Tariq Abdul-Wahad,1998,2003,F,6-6,223.0,1974-11-03,San Jose State University,/players/a/abdulta01.html,abdulta01,0,78.0,[http://www.basketball-reference.com/players/a...


In [25]:
is_list = player_df.cbb_info.apply(isinstance, args=(list,))

In [29]:
len(player_df.loc[is_list, "cbb_info"])

1968

In [30]:
player_df.loc[is_list, "cbb_info"] = ""

In [31]:
player_df2 = player_df.loc[:, ["Bref_ID", "cbb_info"]]

In [33]:
player_df2.rename(columns={"cbb_info": "Sref_Cbb_ID"}, inplace=True)

In [35]:
player_df2["Sref_Cbb_ID"] = player_df2.Sref_Cbb_ID.str.extract("/.*/.*/(.*)\.", 
                                                               expand=False)

In [36]:
player_df2.to_csv("processed_data/sref_cbb_ids_07_07_17.csv", index=False)